In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import string as st

In [2]:
# import Dataset

df=pd.read_csv('D:\\jupyter\\Womens Clothing E-Commerce Reviews.csv',index_col=0)
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [33]:
# Import data to google colab

from google.colab import files
import io

uploaded=files.upload()
df1 = pd.read_csv(io.StringIO(uploaded['Womens Clothing E-Commerce Reviews.csv'].decode('utf-8')),index_col=0)

Saving Womens Clothing E-Commerce Reviews.csv to Womens Clothing E-Commerce Reviews (3).csv


In [0]:
df=pd.DataFrame.copy(df1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23486 entries, 0 to 23485
Data columns (total 10 columns):
Clothing ID                23486 non-null int64
Age                        23486 non-null int64
Title                      19676 non-null object
Review Text                22641 non-null object
Rating                     23486 non-null int64
Recommended IND            23486 non-null int64
Positive Feedback Count    23486 non-null int64
Division Name              23472 non-null object
Department Name            23472 non-null object
Class Name                 23472 non-null object
dtypes: int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Replacing Nan values with mode

j=['Class Name','Department Name','Division Name']
for i in j:
    df[i]=df[i].fillna(df[i].mode()[0])

# Cleaning Nans in Text columns
df['Title']=df['Title'].fillna('.')
df['Review Text']=df['Review Text'].fillna('.')

In [5]:
# Encoding categorical data

def dummy(df,i):
    col=pd.get_dummies(df[i],prefix=i)
    col=col.drop(col.columns[0],axis=1)
    df=pd.concat([df,col],axis=1)
    df=df.drop(i,axis=1)
    return df

j=['Division Name','Department Name','Class Name']
for i in j:
    df=dummy(df,i)    

In [6]:
# Removing Columns With low correlation

x=df.corr()
col=df.columns.difference(['Clothing ID','Title','Review Text','Rating'])

for i in col:
    if ((x.loc[i,'Rating']<0.02) and (x.loc[i,'Rating']>-0.02)):
        df=df.drop(i,axis=1)

In [7]:
df.describe()

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Trend,Class Name_Dresses,Class Name_Jeans,Class Name_Trend
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,918.118709,43.198544,4.196032,0.822362,2.535936,0.063953,0.269054,0.073874,0.005067,0.269650,0.048838,0.005067
std,203.298980,12.279544,1.110031,0.382216,5.702202,0.244674,0.443477,0.261571,0.071003,0.443787,0.215533,0.071003
min,0.000000,18.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,861.000000,34.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,936.000000,41.000000,5.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1078.000000,52.000000,5.000000,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1205.000000,99.000000,5.000000,1.000000,122.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
desc=df.describe().T
desc['1%']=df.quantile(.01)
desc['5%']=df.quantile(.05)
desc['95%']=df.quantile(.95)
desc['99%']=df.quantile(.99)
desc['Q3-Q1']=desc['75%']-desc['25%']
desc['UB']=desc['75%']+(desc['Q3-Q1']*1.5)
desc['LB']=desc['25%']-(desc['Q3-Q1']*1.5)
desc=desc.loc[:,['count','mean','std','min','max','1%','5%','25%','50%','75%','95%','99%','Q3-Q1','LB','UB']]
desc

,count,mean,std,min,max,1%,5%,25%,50%,75%,95%,99%,Q3-Q1,LB,UB
Clothing ID,23486.0,918.118709,203.298980,0.0,1205.0,129.0,417.0,861.0,936.0,1078.0,1104.0,1132.0,217.0,535.5,1403.5
Age,23486.0,43.198544,12.279544,18.0,99.0,21.0,25.0,34.0,41.0,52.0,65.0,74.0,18.0,7.0,79.0
Rating,23486.0,4.196032,1.110031,1.0,5.0,1.0,2.0,4.0,5.0,5.0,5.0,5.0,1.0,2.5,6.5
Recommended IND,23486.0,0.822362,0.382216,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Positive Feedback Count,23486.0,2.535936,5.702202,0.0,122.0,0.0,0.0,0.0,1.0,3.0,11.0,26.0,3.0,-4.5,7.5
Division Name_Initmates,23486.0,0.063953,0.244674,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Department Name_Dresses,23486.0,0.269054,0.443477,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,-1.5,2.5
Department Name_Intimate,23486.0,0.073874,0.261571,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Department Name_Trend,23486.0,0.005067,0.071003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Class Name_Dresses,23486.0,0.269650,0.443787,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,-1.5,2.5


In [9]:
# Feature Engineering

def count_punct(text):
    count=sum([1 for char in text if char in st.punctuation])
    return round(count/(len(text)-text.count(' ')),3)*100

df['Review Length']=df['Review Text'].apply(lambda x: len(x)-x.count(' '))
df['punct%']=df['Review Text'].apply(lambda x:count_punct(x))

In [10]:
# Handling Outliers

from sklearn.preprocessing  import MinMaxScaler

mm=MinMaxScaler(feature_range=(0,1))
df['Age']=mm.fit_transform(df[['Age']])
df['Positive Feedback Count']=mm.fit_transform(df[['Positive Feedback Count']])
df['Review Length']=mm.fit_transform(df[['Review Length']])
df['punct%']=mm.fit_transform(df[['punct%']])

In [11]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Trend,Class Name_Dresses,Class Name_Jeans,Class Name_Trend,Review Length,punct%
0,767,0.185185,.,Absolutely wonderful - silky and sexy and comf...,4,1,0.000000,1,0,1,0,0,0,0,0.105882,0.022
1,1080,0.197531,.,Love this dress! it's sooo pretty. i happene...,5,1,0.032787,0,1,0,0,1,0,0,0.555294,0.055
2,1077,0.518519,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0.000000,0,1,0,0,1,0,0,0.945882,0.035
3,1049,0.395062,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0.000000,0,0,0,0,0,0,0,0.240000,0.087
4,847,0.358025,Flattering shirt,This shirt is very flattering to all due to th...,5,1,0.049180,0,0,0,0,0,0,0,0.367059,0.032


In [12]:
# Combining Title and Review for analysis

df['Review']=df['Title']+' '+df['Review Text']

In [13]:
# Removing Punctuation

def rem_punct(txt):
    txt=''.join([i.lower() for i in txt if i not in st.punctuation])
    return txt

df['Review_cln']=df['Review'].apply(lambda x:rem_punct(x))

In [14]:
# Rare word removal

freq = pd.Series(' '.join(df['Review_cln']).split()).value_counts()
freq = freq[freq<5]
freq=freq.index

df['Review_cln']=df['Review_cln'].apply(lambda x: ' '.join([i for i in x.split() if i not in freq]))

In [15]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,Department Name_Trend,Class Name_Dresses,Class Name_Jeans,Class Name_Trend,Review Length,punct%,Review,Review_cln
0,767,0.185185,.,Absolutely wonderful - silky and sexy and comf...,4,1,0.000000,1,0,1,0,0,0,0,0.105882,0.022,. Absolutely wonderful - silky and sexy and co...,absolutely wonderful silky and sexy and comfor...
1,1080,0.197531,.,Love this dress! it's sooo pretty. i happene...,5,1,0.032787,0,1,0,0,1,0,0,0.555294,0.055,. Love this dress! it's sooo pretty. i happe...,love this dress its sooo pretty i happened to ...
2,1077,0.518519,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0.000000,0,1,0,0,1,0,0,0.945882,0.035,Some major design flaws I had such high hopes ...,some major design flaws i had such high hopes ...
3,1049,0.395062,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0.000000,0,0,0,0,0,0,0,0.240000,0.087,"My favorite buy! I love, love, love this jumps...",my favorite buy i love love love this jumpsuit...
4,847,0.358025,Flattering shirt,This shirt is very flattering to all due to th...,5,1,0.049180,0,0,0,0,0,0,0,0.367059,0.032,Flattering shirt This shirt is very flattering...,flattering shirt this shirt is very flattering...


In [16]:
# Basic Text Operations (tokenizing,lemmatization)

sw=nltk.corpus.stopwords.words('english')
wn=nltk.WordNetLemmatizer()

def op(txt):
    txt=re.split('\W+',txt)
    txt=[i for i in txt if i not in sw ]
    txt=[wn.lemmatize(i) for i in txt]
    txt=[i for i in txt if i.isalpha()]
    txt=[i for i in txt if len(i)>2]
    return txt

df['Review_cln']=df['Review_cln'].apply(lambda x:op(x))

In [18]:
# Finding sentiment of review

from textblob import TextBlob

bloblist=list()

for row in df['Review_cln'].astype(str):
    blob=TextBlob(row)
    bloblist.append((row,blob.sentiment.polarity,blob.subjectivity))
    df_senti=pd.DataFrame(bloblist,columns=['Row','Setiment','Polarity'])
  
df=pd.concat([df,df_senti],axis=1)
df.drop('Row',axis=1,inplace=True)

In [19]:
# Converting back to sentence for tf-idf

def joinin(txt):
    txt=' '.join([i for i in txt])
    return txt

df['Review_cln']=df['Review_cln'].apply(lambda x:joinin(x))

In [20]:
df.loc[:15,'Review_cln']

0           absolutely wonderful silky sexy comfortable
1     love dress sooo pretty happened find store gla...
2     major design flaw high hope dress really wante...
3     favorite buy love love love jumpsuit fun flirt...
4     flattering shirt shirt flattering due adjustab...
5     petite love tracy reese dress one petite foot ...
6     shimmer fun basket hte last see would look lik...
7     shimmer surprisingly lot ordered carbon store ...
8     flattering love dress usually get run little s...
9     fun dress ordered petite make sure length wasn...
10    dress look like made cheap material dress run ...
11                   dress perfection pretty flattering
12    perfect find review written shopper past right...
13    run big bought black midi dress didnt bother l...
14    pretty party dress issue nice choice holiday l...
15    nice body took package wanted fit badly could ...
Name: Review_cln, dtype: object

In [21]:
# Using Tf-idf vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
tf_vect=TfidfVectorizer()

In [22]:
rt_tf_vect=tf_vect.fit_transform(df['Review_cln'])
print(rt_tf_vect.shape)
print(tf_vect.get_feature_names())

(23486, 4277)
['abdomen', 'ability', 'able', 'absolute', 'absolutely', 'abstract', 'abt', 'abundance', 'accent', 'accented', 'accentuate', 'accentuated', 'accentuates', 'accentuating', 'acceptable', 'access', 'accessorize', 'accessorized', 'accessory', 'accident', 'accidentally', 'accommodate', 'accommodates', 'according', 'account', 'accurate', 'accurately', 'achieve', 'across', 'acrylic', 'act', 'active', 'activity', 'actual', 'actuality', 'actually', 'add', 'added', 'addicted', 'adding', 'addition', 'additional', 'additionally', 'adequate', 'adequately', 'adjust', 'adjustable', 'adjusted', 'adjusting', 'adjustment', 'admire', 'admired', 'admiring', 'admit', 'admittedly', 'adn', 'adorable', 'adore', 'adored', 'adult', 'advantage', 'advertised', 'advice', 'advise', 'advised', 'aesthetic', 'aesthetically', 'affect', 'afford', 'affordable', 'afraid', 'afternoon', 'age', 'aged', 'ago', 'agree', 'agreed', 'ahead', 'ahve', 'air', 'airdrying', 'airy', 'aka', 'akemi', 'ala', 'albeit', 'align

In [23]:
#Vectorizing Sparse matrix for review

df_vect_rt=pd.DataFrame(rt_tf_vect.toarray())
df_vect_rt.columns=tf_vect.get_feature_names()
df_vect_rt.head()

,abdomen,ability,able,absolute,absolutely,abstract,abt,abundance,accent,accented,...,yuck,yummy,zero,zip,zipped,zipper,zippered,zipping,zone,zoom
0,0.0,0.0,0.0,0.0,0.386812,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.135291,0.0,0.11832,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0


In [24]:
# Concatinating Entire data

df=pd.concat([df,df_vect_rt],axis=1)
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name_Initmates,Department Name_Dresses,Department Name_Intimate,...,yuck,yummy,zero,zip,zipped,zipper,zippered,zipping,zone,zoom
0,767,0.185185,.,Absolutely wonderful - silky and sexy and comf...,4,1,0.000000,1,0,1,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
1,1080,0.197531,.,Love this dress! it's sooo pretty. i happene...,5,1,0.032787,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
2,1077,0.518519,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0.000000,0,1,0,...,0.0,0.0,0.0,0.135291,0.0,0.11832,0.0,0.0,0.0,0.0
3,1049,0.395062,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0.000000,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0
4,847,0.358025,Flattering shirt,This shirt is very flattering to all due to th...,5,1,0.049180,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0


In [25]:
# Separting output variable

y=df['Rating']
df=df.drop('Rating',axis=1)
df=df.set_index('Clothing ID')

In [26]:
#Removing text

df=df.drop(['Title','Review Text','Review_cln','Review'],axis=1)

In [28]:
# Over-sampling using SMOTE 4 time for balacing the entire dataset

from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=0,k_neighbors=5,ratio='minority')

df,y=sm.fit_sample(df,y)

df,y=sm.fit_sample(df,y)

df,y=sm.fit_sample(df,y)

df,y=sm.fit_sample(df,y)

In [29]:
df.shape

(65655, 4291)

In [30]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{1: 13131, 2: 13131, 3: 13131, 4: 13131, 5: 13131}

In [31]:
# Label encoding output variable

col=pd.get_dummies(y,prefix='Rating')

In [32]:
# Splitting data into training and testing

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df,col,test_size=0.2)

__Model: KNN__

In [75]:
from sklearn.neighbors import KNeighborsClassifier

model3=KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
model3.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [0]:
# Finding the score for validation

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score,accuracy_score

tr_pred=model3.predict(x_train)
y_pred = model3.predict(x_test)


trprecision,trrecall,trfscore,trsupport=score(y_train,tr_pred)
tracc=accuracy_score(y_train,tr_pred)
precision,recall,fscore,support=score(y_test,y_pred)
acc=accuracy_score(y_test,y_pred)

In [78]:
# For Training

print('Precision : ',trprecision)
print('\nRecall : ',trrecall)
print('\nF-Score :',trfscore)
print('\nAccuracy : ',tracc)

Precision :  [0.97818761 0.97211118 0.86706231 0.63190342 0.85253456]

Recall :  [0.99580673 0.99467326 0.97557951 0.9343842  0.08806169]

F-Score : [0.98691854 0.98326281 0.9181255  0.75393633 0.15963414]

Accuracy :  0.7976924834361435


In [79]:
# For Testing

print('Precision : ',precision)
print('\nRecall : ',recall)
print('\nF-Score :',fscore)
print('\nAccuracy : ',acc)

Precision :  [0.96678967 0.95533407 0.80688767 0.56159687 0.75369458]

Recall :  [0.99317665 0.98854087 0.94675227 0.88192308 0.05824134]

F-Score : [0.97980553 0.97165384 0.8712424  0.68621876 0.10812721]

Accuracy :  0.7738176833447566
